In [1]:
from custom_dataset import Dataset_Assay, Dataset_Assay_Survival, custom_collate_assay, custom_collate_assay_survival
from models import *
from saveAndLoad import *

# LOAD DATA
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)

device = 'cuda:1'

mut_embeddings = np.load('../aa/canonical_mut_average_embeddings_esm.npy')
ref_embeddings = np.load('../aa/canonical_ref_embeddings_esm2.npy')
tumors = pickleLoad('../aa/tumors.pkl')
assays = pickleLoad('../aa/assays.pkl')

/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


0 	 data_0_00_survival_ratio1.csv
1 	 data_0_00percentMinCancerType.csv
2 	 data_0_10percentMinCancerType.csv
5 	 data_1_00_survival_ratio1.csv
4 	 data_1_00percentMinCancerType.csv
3 	 label_mappings
loading data from ../aa/tumors.pkl
loading data from ../aa/assays.pkl


In [5]:
data_emb = labeled_data[4]
print(data_emb)
data_df_emb = pd.read_csv(data_dir+data_emb)
nlabels = len(data_df_emb['CANCER_TYPE_INT'].unique())
print('n labels:',nlabels)

dataset = Dataset_Assay(data_df_emb, 'CANCER_TYPE_INT', mut_embeddings, ref_embeddings, tumors, assays, device)
train_loader, test_loader = getTrainTestLoaders(dataset, batch_size = 750, collate=custom_collate_assay)

data_1_00percentMinCancerType.csv
n labels: 20
145508 samples
131202 unique patients
20 labels


In [6]:
## ATTENTION MODEL WITH POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 1448
    position_embedding: bool = False
    num_heads: int = 1

config_att = Config_Att()
config_att.n_labels = nlabels

model = Classifier(config_att)
model.to(device)

num_epochs = 15
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.001)
train_assay(model,num_epochs,train_loader,test_loader, criterion, optimizer)

TESTING: 100%|██████████| 39/39 [01:05<00:00,  1.67s/it]


Test Accuracy: 41.06%, (11949 of 29102)


TESTING: 100%|██████████| 39/39 [01:05<00:00,  1.67s/it]


Test Accuracy: 50.23%, (14619 of 29102)


TESTING: 100%|██████████| 39/39 [01:05<00:00,  1.67s/it]


...

Test Accuracy: 57.59%, (16759 of 29102)


TESTING: 100%|██████████| 39/39 [01:08<00:00,  1.75s/it]


Test Accuracy: 58.06%, (16898 of 29102)


TESTING: 100%|██████████| 39/39 [01:06<00:00,  1.70s/it]

Test Accuracy: 58.19%, (16934 of 29102)
Best Accuracy: 58.19% at epoch 14


In [3]:
data_emb = labeled_data[0]
print(data_emb)
data_df_emb = pd.read_csv(data_dir+data_emb)
dataset = Dataset_Assay_Survival(data_df_emb,mut_embeddings, ref_embeddings, tumors, assays, device)
train_loader, test_loader = getTrainTestLoaders(dataset, batch_size = 500, collate=custom_collate_assay_survival)

data_0_00_survival_ratio1.csv
43332 samples
41544 unique patients
89 cancer types
620 detailed cancer types


In [4]:
## ATTENTION MODEL WITH POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 1
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 1448
    position_embedding: bool = False
    num_heads: int = 1

config_att = Config_Att()

model = Classifier(config_att)
model.to(device)

num_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=.00001)
criterion = negative_log_partial_likelihood
# train_assay_survival(model,num_epochs,train_loader,test_loader, criterion, optimizer, saveName = './best_models/model_sha_survival_uncensored.pt')
train_assay_survival(model,num_epochs,train_loader,test_loader, criterion, optimizer)

TESTING: 100%|██████████| 18/18 [00:19<00:00,  1.07s/it]


C-Index: 0.7527


TESTING: 100%|██████████| 18/18 [00:19<00:00,  1.07s/it]


C-Index: 0.7543


TESTING: 100%|██████████| 18/18 [00:19<00:00,  1.07s/it]


C-Index: 0.7520


TESTING: 100%|██████████| 18/18 [00:19<00:00,  1.06s/it]


C-Index: 0.7571


TESTING: 100%|██████████| 18/18 [00:19<00:00,  1.06s/it]

C-Index: 0.7538
Best C-Index: 0.7571 at epoch 3
